In [1]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    # 단일 URL 크롤링: requests로 가져와 newspaper 파싱 (403시 /amp 폴백)
    try:
        r = requests.get(url, headers=HEADERS, timeout=12)
        if r.status_code == 403:
            r = requests.get(url.rstrip("/") + "/amp", headers=HEADERS, timeout=12)
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None
        return {"url": url, "title": art.title, "text": art.text, "publish_date": art.publish_date}
    except:
        return None

def crawl_g_articles(date_str, max_workers=31):
    """g 기사 크롤링 (0000~9999)"""
    urls = [f"{BASE_URL}{date_str}{num:04}g" for num in range(10000)]
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futures = [ex.submit(crawl_url, u) for u in urls]
        collected = 0
        with tqdm(total=len(futures), desc=f"g {date_str}") as pbar:
            for fut in as_completed(futures):
                res = fut.result()
                if res:
                    results.append(res)
                    collected += 1
                pbar.set_postfix_str(f"수집 {collected}") # 현재 수집 건수
                pbar.update(1)

                time.sleep(0.06)  # 너무 빠른 요청 방지(차단 완화)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    tqdm.write(f"최종 수집 기사 수: {len(df)}") # 최종 수집 건 수
    return df

In [2]:
day_str = "20250820"

In [3]:
out_df = crawl_g_articles(day_str)
out_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_g_{day_str}.csv", index=False)

out_df

g 20250820: 100%|██████████| 10000/10000 [11:01<00:00, 15.12it/s, 수집 15]

최종 수집 기사 수: 15


,url,title,text,publish_date
0,https://www.hankyung.com/article/202508200011g,"이스타항공, 부산발 국제선 늘린다…오사카·후쿠오카·삿포로 취항","사진=이스타항공\n\n이스타항공은 10월 26일부터 부산발 오사카, 후쿠오카, 삿포...",2025-08-20 08:59:43+09:00
1,https://www.hankyung.com/article/202508200082g,"BYD코리아, '씰 AWD' 본격 판매 시작…국가 보조금 178만원",씰./사진=BYD코리아\n\n비야디(BYD)코리아는 자사의 두번째 국내 승용 모델 ...,2025-08-20 09:20:40+09:00
2,https://www.hankyung.com/article/202508200053g,'사랑꾼'들 잔뜩 몰렸다…포털 최상단 개방했더니 '깜짝',사용자 참여형 서비스 '네이트 오늘'\n\n일평균 '140시간' 유료 구좌 운영\n...,2025-08-20 10:50:58+09:00
3,https://www.hankyung.com/article/202508200209g,"현대차, 2025 세계경제학자대회에 수소차 신형 넥쏘 전시",세계계량경제학회 주최로 5년마다 개최\n\n18~22일 코엑스에서 진행\n\n서울 ...,2025-08-20 09:40:36+09:00
4,https://www.hankyung.com/article/202508200262g,지드래곤도 탔는데…'100만대 예약' 테슬라 사이버트럭의 굴욕,"사이버트럭 美 판매량 5만2000대 그쳐\n\n""사전 계약 100만대"" 대비 저조한...",2025-08-20 10:24:45+09:00
5,https://www.hankyung.com/article/202508200138g,"하나투어ITC, 네이버 인바운드 여행 공식 예약 파트너사로 선정",사진=하나투어\n\n하나투어ITC는 네이버와 인바운드 여행 비즈니스 확대를 위한 전...,2025-08-20 09:31:26+09:00
6,https://www.hankyung.com/article/202508200571g,"신세계인터내셔날, 신규 여성복 '자아(JAAH)' 선봬",젊고 활동적인 여성 겨낭\n\n사진=신세계인터내셔날 제공\n\n신세계인터내셔날이 여...,2025-08-20 11:36:30+09:00
7,https://www.hankyung.com/article/202508200661g,"中신생업체도 韓시장 '군침'…""로청 성공에 안방가전 각축""","모바, 20일 한국 진출 공식 선언\n\n로봇청소기 등 생활가전 대거 공개\n\n'...",2025-08-20 13:05:00+09:00
8,https://www.hankyung.com/article/202508200666g,"농심, 케데헌 입은 신라면·새우깡 출시…""글로벌 팬 공략""",사진=농심 제공\n\n농심이 넷플릭스 인기 애니메이션 ‘케이팝 데몬 헌터스’(케데헌...,2025-08-20 12:06:38+09:00
9,https://www.hankyung.com/article/202508209858g,"美 정부 '삼성전자 주주' 되나…""인텔 이어 지분 인수 검토""","로이터 ""美 상무부, 삼성 등 지분 인수 검토""\n\n인텔 지분 10% 인수 등 투...",2025-08-20 09:19:35+09:00
